In [ ]:
from nbfix import load_catalog

from typing import Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display
from matplotlib.ticker import AutoMinorLocator, MultipleLocator


In [ ]:
catalog = load_catalog()

df = pd.concat(
    [
        catalog.load("train.cycle"),
        catalog.load("train.clean"),
        catalog.load("train.velocity"),
        catalog.load("train.rolling"),
        catalog.load("train.log"),
        catalog.load("train.sqrt"),
        catalog.load("train.pressure"),
        catalog.load("model_ridge_out").rename(columns={"pa": "pred"}),
    ],
    axis=1,
)
display(df)

num_cycles = df["cycle"].max()


In [ ]:
def plot_cycle(n: int, figsize: Tuple[int, int] = (20, 10)):
    c = df[df["cycle"] == n]

    # get a multi-axes figure
    fig, ax1 = plt.subplots(figsize=figsize)
    fig.subplots_adjust(right=0.75)
    ax2, ax3, ax4, ax5 = ax1.twinx(), ax1.twinx(), ax1.twinx(), ax1.twinx()
    ax3.spines.right.set_position(("axes", 1.07))
    ax4.spines.right.set_position(("axes", 1.14))
    ax5.spines.right.set_position(("axes", 1.21))

    # plot the data
    ax1.plot(c.phi, c.pa, c="green", label="pressure", lw=3)
    ax1.plot(c.phi, c.pred, c="green", label="prediction", ls=":", lw=2, alpha=0.5)
    ax1.set_ylabel("pressure")

    ax2.plot(c.phi, c.acc, c="gray", label="acceleration", ls=":", lw=0, marker=".")
    ax2.set_ylabel("acceleration")

    # ax3.plot(c.phi, c.vel, c="gold", label="velocity")
    # ax3.set_ylabel("velocity")

    ax4.plot(c.phi, c.vel_abs, c="darkorange", label="velocity (absolute)")
    ax4.set_ylabel("velocity (absolute)")

    ax5.plot(c.phi, c.roll_100_std, c="aqua", ls="-", label="rolling std 100")
    # ax5.plot(c.phi, c.roll_25_std, c="fuchsia", ls="-", label="rolling std 25")
    # ax5.plot(c.phi, c.roll_50_std, c="coral", ls="-", label="rolling std 50")
    ax5.set_ylabel("rolling std")

    # place dots at maximum
    pa_max_xy = c.phi.iloc[c.pa.argmax()], c.pa.max()
    ax1.plot(*pa_max_xy, c="green", marker="v", markersize=10)

    vel_max_xy = c.phi.iloc[c.acc.abs().argmax()], c.acc.abs().max()
    ax2.plot(*vel_max_xy, c="gray", marker="^", markersize=10)

    vel_max_xy2 = c.phi.iloc[c.vel.argmax()], c.vel.max()
    ax3.plot(*vel_max_xy2, c="gold", marker="^", markersize=10)

    # grids
    ax1.xaxis.set_major_locator(MultipleLocator(10))
    ax1.xaxis.set_minor_locator(AutoMinorLocator(5))

    ax1.grid(which='major', color='#CCCCCC', linestyle='--')
    ax1.grid(which='minor', color='#CCCCCC', linestyle=':')
    
    # labels, titles
    ax1.set_xlabel("crank angle")
    ax1.set_title(f"cycle {n}")

    fig.legend(loc="upper center", ncol=len(fig.axes))

    plt.show()

In [ ]:
for i in np.random.randint(1, num_cycles + 1, size=5):
    plot_cycle(i)